In [49]:
""" how to use polars
    auther:  ryoma yokoyama
    created: 2025.05.13 13:12:50
"""
import os
import sys
import time
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib_fontja
import numpy as np
import polars as pl
import seaborn as sns
import tqdm
from hydra import compose, initialize
from omegaconf import OmegaConf

from src.seed import seed_everything

with initialize(config_path="config", version_base=None):
    cfg = compose(config_name="config.yaml")
    cfg.exp_number = Path().resolve().name
print(OmegaConf.to_yaml(cfg, resolve=True))

seed_everything(cfg.seed)

warnings.filterwarnings("ignore")


exp_number: '000'
data:
  data_root: ../../data
  results_root: ../../results
  train_path: ../../data/train.csv
  test_path: ../../data/test.csv
  sample_submission_path: ../../data/sample_submission.csv
  results_dir: ../../results/000
seed: 42
n_splits: 5
target: SalePrice



目的変数（予測対象）： SalePrice  

データ：
* train.csv: モデルの学習に使用するデータ（販売価格SalePriceを含む）
* test.csv: 販売価格を予測する対象のデータ（SalePriceは含まない）
* data_description.txt: 各特徴量の詳細な説明
* sample_submission.csv: 提出ファイルの形式例

## polarsの機能


`read_csv`

`pands`の`read_csv`と異なり、型定義を自動で行う。

polarsは`infer_schema`がデフォルトで`True`になっているため、型を自動で推測してくれる
`infer_schema_length`で推測する行数を指定できる(デフォルトは100)



* `schema` :各列の名前とデータ型を事前に定義
* `schema_overrides` : 各列のデータ型を上書きする
* `null_values` : nullとみなす値を指定する
* `try_parse_dates` : 日付型を自動で推測するかどうか

In [ ]:
train_df = pl.read_csv(cfg.data.train_path, infer_schema_length=10_000)

# train_dfから型を取得
train_schema = train_df.schema

# ComputeError: could not parse `NA` as dtype `i64`が出るためnull_valuesを指定
test_df = pl.read_csv(
    cfg.data.test_path,
    infer_schema_length=10_000,
    schema_overrides=train_schema,
    null_values=["NA"],
)


### 列の操作
 `select`
特定の列を選択する
* `pl.col` : 列を選択する
* `pl.all` : 全ての列を選択する
* `pl.exclude` : 特定の列を除外する
* `pl.first` : 最初の列を選択する
* `pl.last` : 最後の列を選択する
* `pl.head` : 最初のn行を選択する
* `pl.tail` : 最後のn行を選択する
* `pl.slice` : 特定の行を選択する

In [20]:
# 1個の列を選択。train_df.select(pl.col("MSSubClass"))でも
display(train_df.select("MSSubClass").head(1))
# 複数の列を選択。train_df.select(pl.col("MSSubClass"), pl.col("MSZoning"))でも
display(train_df.select(["MSSubClass", "MSZoning"]).head(1))
# 特定の列を除外して選択
display(train_df.select(pl.exclude("MSSubClass")).head(1))
# 最初の列を選択
display(train_df.select(pl.first()).head(1))
# 最後の列を選択
display(train_df.select(pl.last()).head(1))


MSSubClass
i64
60


MSSubClass,MSZoning
i64,str
60,"""RL"""


Id,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,…,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,str,str,str,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,i64
1,"""RL""","""65""",8450,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""CollgCr""","""Norm""","""Norm""","""1Fam""","""2Story""",7,5,2003,2003,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""","""196""","""Gd""","""TA""","""PConc""","""Gd""","""TA""","""No""","""GLQ""",706,"""Unf""",0,150,…,854,0,1710,1,0,2,1,3,1,"""Gd""",8,"""Typ""",0,"""NA""","""Attchd""","""2003""","""RFn""",2,548,"""TA""","""TA""","""Y""",0,61,0,0,0,0,"""NA""","""NA""","""NA""",0,2,2008,"""WD""","""Normal""",208500


Id
i64
1


SalePrice
i64
208500


### 行の操作
`filter`
特定の条件を満たす行を選択する  
`sort`
特定の列でソートする

In [ ]:
# filetr
display(train_df.filter(pl.col("MSSubClass") == 20).head(1))

# 複数条件でfilter
display(train_df.filter((pl.col("MSSubClass") == 20) & (pl.col("MSZoning") == "RL")).head(1))
display(train_df.filter((pl.col("MSSubClass") == 20) | (pl.col("MSZoning") == "RL")).head(1))

# sliceで複数行を取得
display(train_df.slice(offset=0, length=5))
display(train_df[0:5])


Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,…,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
i64,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,str,str,str,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,i64
2,20,"""RL""","""80""",9600,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""FR2""","""Gtl""","""Veenker""","""Feedr""","""Norm""","""1Fam""","""1Story""",6,8,1976,1976,"""Gable""","""CompShg""","""MetalSd""","""MetalSd""","""None""","""0""","""TA""","""TA""","""CBlock""","""Gd""","""TA""","""Gd""","""ALQ""",978,"""Unf""",0,…,0,0,1262,0,1,2,0,3,1,"""TA""",6,"""Typ""",1,"""TA""","""Attchd""","""1976""","""RFn""",2,460,"""TA""","""TA""","""Y""",298,0,0,0,0,0,"""NA""","""NA""","""NA""",0,5,2007,"""WD""","""Normal""",181500


Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,…,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
i64,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,str,str,str,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,i64
2,20,"""RL""","""80""",9600,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""FR2""","""Gtl""","""Veenker""","""Feedr""","""Norm""","""1Fam""","""1Story""",6,8,1976,1976,"""Gable""","""CompShg""","""MetalSd""","""MetalSd""","""None""","""0""","""TA""","""TA""","""CBlock""","""Gd""","""TA""","""Gd""","""ALQ""",978,"""Unf""",0,…,0,0,1262,0,1,2,0,3,1,"""TA""",6,"""Typ""",1,"""TA""","""Attchd""","""1976""","""RFn""",2,460,"""TA""","""TA""","""Y""",298,0,0,0,0,0,"""NA""","""NA""","""NA""",0,5,2007,"""WD""","""Normal""",181500


Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,…,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
i64,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,str,str,str,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,i64
1,60,"""RL""","""65""",8450,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""CollgCr""","""Norm""","""Norm""","""1Fam""","""2Story""",7,5,2003,2003,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""","""196""","""Gd""","""TA""","""PConc""","""Gd""","""TA""","""No""","""GLQ""",706,"""Unf""",0,…,854,0,1710,1,0,2,1,3,1,"""Gd""",8,"""Typ""",0,"""NA""","""Attchd""","""2003""","""RFn""",2,548,"""TA""","""TA""","""Y""",0,61,0,0,0,0,"""NA""","""NA""","""NA""",0,2,2008,"""WD""","""Normal""",208500


Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,…,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
i64,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,str,str,str,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,i64
1,60,"""RL""","""65""",8450,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""CollgCr""","""Norm""","""Norm""","""1Fam""","""2Story""",7,5,2003,2003,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""","""196""","""Gd""","""TA""","""PConc""","""Gd""","""TA""","""No""","""GLQ""",706,"""Unf""",0,…,854,0,1710,1,0,2,1,3,1,"""Gd""",8,"""Typ""",0,"""NA""","""Attchd""","""2003""","""RFn""",2,548,"""TA""","""TA""","""Y""",0,61,0,0,0,0,"""NA""","""NA""","""NA""",0,2,2008,"""WD""","""Normal""",208500
2,20,"""RL""","""80""",9600,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""FR2""","""Gtl""","""Veenker""","""Feedr""","""Norm""","""1Fam""","""1Story""",6,8,1976,1976,"""Gable""","""CompShg""","""MetalSd""","""MetalSd""","""None""","""0""","""TA""","""TA""","""CBlock""","""Gd""","""TA""","""Gd""","""ALQ""",978,"""Unf""",0,…,0,0,1262,0,1,2,0,3,1,"""TA""",6,"""Typ""",1,"""TA""","""Attchd""","""1976""","""RFn""",2,460,"""TA""","""TA""","""Y""",298,0,0,0,0,0,"""NA""","""NA""","""NA""",0,5,2007,"""WD""","""Normal""",181500
3,60,"""RL""","""68""",11250,"""Pave""","""NA""","""IR1""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""CollgCr""","""Norm""","""Norm""","""1Fam""","""2Story""",7,5,2001,2002,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""","""162""","""Gd""","""TA""","""PConc""","""Gd""","""TA""","""Mn""","""GLQ""",486,"""Unf""",0,…,866,0,1786,1,0,2,1,3,1,"""Gd""",6,"""Typ""",1,"""TA""","""Attchd""","""2001""","""RFn""",2,608,"""TA""","""TA""","""Y""",0,42,0,0,0,0,"""NA""","""NA""","""NA""",0,9,2008,"""WD""","""Normal""",223500
4,70,"""RL""","""60""",9550,"""Pave""","""NA""","""IR1""","""Lvl""","""AllPub""","""Corner""","""Gtl""","""Crawfor""","""Norm""","""Norm""","""1Fam""","""2Story""",7,5,1915,1970,"""Gable""","""CompShg""","""Wd Sdng""","""Wd Shng""","""None""","""0""","""TA""","""TA""","""BrkTil""","""TA""","""Gd""","""No""","""ALQ""",216,"""Unf""",0,…,756,0,1717,1,0,1,0,3,1,"""Gd""",7,"""Typ""",1,"""Gd""","""Detchd""","""1998""","""Unf""",3,642,"""TA""","""TA""","""Y""",0,35,272,0,0,0,"""NA""","""NA""","""NA""",0,2,2006,"""WD""","""Abnorml""",140000
5,60,"""RL""","""84""",14260,"""Pave""","""NA""","""IR1""","""Lvl""","""AllPub""","""FR2""","""Gtl""","""NoRidge""","""Norm""","""Norm""","""1Fam""","""2Story""",8,5,2000,2000,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""","""350""","""Gd""","""TA""","""PConc""","""Gd""","""TA""","""Av""","""GLQ""",655,"""Unf""",0,…,1053,0,2198,1,0,2,1,4,1,"""Gd""",9,"""Typ""",1,"""TA""","""Attchd""","""2000""","""RFn""",3,836,"""TA""","""TA""","""Y""",192,84,0,0,0,0,"""NA""","""NA""","""NA""",0,12,2008,"""WD""","""Normal""",250000


Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,…,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
i64,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,str,str,str,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,i64
1,60,"""RL""","""65""",8450,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""CollgCr""","""Norm""","""Norm""","""1Fam""","""2Story""",7,5,2003,2003,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""","""196""","""Gd""","""TA""","""PConc""","""Gd""","""TA""","""No""","""GLQ""",706,"""Unf""",0,…,854,0,1710,1,0,2,1,3,1,"""Gd""",8,"""Typ""",0,"""NA""","""Attchd""","""2003""","""RFn""",2,548,"""TA""","""TA""","""Y""",0,61,0,0,0,0,"""NA""","""NA""","""NA""",0,2,2008,"""WD""","""Normal""",208500
2,20,"""RL""","""80""",9600,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""FR2""","""Gtl""","""Veenker""","""Feedr""","""Norm""","""1Fam""","""1Story""",6,8,1976,1976,"""Gable""","""CompShg""","""MetalSd""","""MetalSd""","""None""","""0""","""TA""","""TA""","""CBlock""","""Gd""","""TA""","""Gd""","""ALQ""",978,"""Unf""",0,…,0,0,1262,0,1,2,0,3,1,"""TA""",6,"""Typ""",1,"""TA""","""Attchd""","""1976""","""RFn""",2,460,"""TA""","""TA""","""Y""",298,0,0,0,0,0,"""NA""","""NA""","""NA""",0,5,2007,"""WD""","""Normal""",181500
3,60,"""RL""","""68""",11250,"""Pave""","""NA""","""IR1""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""CollgCr""","""Norm""","""Norm""","""1Fam""","""2Story""",7,5,2001,2002,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""","""162""","""Gd""","""TA""","""PConc""","""Gd""","""TA""","""Mn""","""GLQ""",486,"""Unf""",0,…,866,0,1786,1,0,2,1,3,1,"""Gd""",6,"""Typ""",1,"""TA""","""Attchd""","""2001""","""RFn""",2,608,"""TA""","""TA""","""Y""",0,42,0,0,0,0,"""NA""","""NA""","""NA""",0,9,2008,"""WD""","""Normal""",223500
4,70,"""RL""","""60""",9550,"""Pave""","""NA""","""IR1""","""Lvl""","""AllPub""","""Corner""","""Gtl""","""Crawfor""","""Norm""","""Norm""","""1Fam""","""2Story""",7,5,1915,1970,"""Gable""","""CompShg""","""Wd Sdng""","""Wd Shng""","""None""","""0""","""TA""","""TA""","""BrkTil""","""TA""","""Gd""","""No""","""ALQ""",216,"""Unf""",0,…,756,0,1717,1,0,1,0,3,1,"""Gd""",7,"""Typ""",1,"""Gd""","""Detchd""","""1998""","""Unf""",3,642,"""TA""","""TA""","""Y""",0,35,272,0,0,0,"""NA""","""NA""","""NA""",0,2,2006,"""WD""","""Abnorml""",140000
5,60,"""RL""","""84""",14260,"""Pave""","""NA""","""IR1""","""Lvl""","""AllPub""","""FR2""","""Gtl""","""NoRidge""","""Norm""","""Norm""","""1Fam""","""2Story""",8,5,2000,2000,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""","""350""","""Gd""","""TA""","""PConc""","""Gd""","""TA""","""Av""","""GLQ""",655,"""Unf""",0,…,1053,0,2198,1,0,2,1,4,1,"""Gd""",9,"""Typ""",1,"""TA""","""Attchd""","""2000""","""RFn""",3,836,"""TA""","""TA""","""Y""",192,84,0,0,0,0,"""NA""","""NA""","""NA""",0,12,2008,"""WD""","""Normal""",250000


In [30]:
# sort
display(train_df.sort("MSSubClass").head(1))
# 降順
display(train_df.sort("MSSubClass", descending=True).head(1))
# 複数
display(train_df.sort(["MSSubClass", "MSZoning"], descending=[True, False]).head(1))


Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,…,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
i64,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,str,str,str,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,i64
2,20,"""RL""","""80""",9600,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""FR2""","""Gtl""","""Veenker""","""Feedr""","""Norm""","""1Fam""","""1Story""",6,8,1976,1976,"""Gable""","""CompShg""","""MetalSd""","""MetalSd""","""None""","""0""","""TA""","""TA""","""CBlock""","""Gd""","""TA""","""Gd""","""ALQ""",978,"""Unf""",0,…,0,0,1262,0,1,2,0,3,1,"""TA""",6,"""Typ""",1,"""TA""","""Attchd""","""1976""","""RFn""",2,460,"""TA""","""TA""","""Y""",298,0,0,0,0,0,"""NA""","""NA""","""NA""",0,5,2007,"""WD""","""Normal""",181500


Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,…,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
i64,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,str,str,str,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,i64
10,190,"""RL""","""50""",7420,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""Corner""","""Gtl""","""BrkSide""","""Artery""","""Artery""","""2fmCon""","""1.5Unf""",5,6,1939,1950,"""Gable""","""CompShg""","""MetalSd""","""MetalSd""","""None""","""0""","""TA""","""TA""","""BrkTil""","""TA""","""TA""","""No""","""GLQ""",851,"""Unf""",0,…,0,0,1077,1,0,1,0,2,2,"""TA""",5,"""Typ""",2,"""TA""","""Attchd""","""1939""","""RFn""",1,205,"""Gd""","""TA""","""Y""",0,4,0,0,0,0,"""NA""","""NA""","""NA""",0,1,2008,"""WD""","""Normal""",118000


Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,…,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
i64,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,str,str,str,str,i64,i64,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,i64
94,190,"""C (all)""","""60""",7200,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""Corner""","""Gtl""","""OldTown""","""Norm""","""Norm""","""2fmCon""","""2.5Unf""",6,6,1910,1998,"""Hip""","""CompShg""","""MetalSd""","""MetalSd""","""None""","""0""","""TA""","""TA""","""BrkTil""","""TA""","""Fa""","""Mn""","""Rec""",1046,"""Unf""",0,…,1031,0,2291,0,1,2,0,4,2,"""TA""",9,"""Typ""",1,"""Gd""","""Detchd""","""1900""","""Unf""",2,506,"""TA""","""TA""","""Y""",0,0,0,0,99,0,"""NA""","""NA""","""NA""",0,11,2007,"""WD""","""Normal""",133900


### 列の追加・変更
`with_columns` : 新しい列を追加したり、既存の列を変更する
* `pl.when` : 条件分岐を行う
* `pl.when().then().otherwise()` : 条件分岐を行う

* `cast` : 列の型を変更する
* `alias` : 列の名前を変更する
* `fill_null` : nullを特定の値で埋める
* `fill_nan` : NaNを特定の値で埋める


`map_elements` : 各要素に対して関数を適用する

In [59]:
train_df = train_df.with_columns((pl.col("YrSold") - pl.col("YearBuilt")).alias("HouseAge"))


HouseAge
i64
-6004
-5897
-5996
-5654
-5992
…
-5989
-5902
-5754


In [33]:
# pl.col("YrSold") - pl.col("YearBuilt"): 売り出し年 - 建築年 = 築年数によって
train_df = train_df.with_columns(
    pl.when((pl.col("YrSold") - pl.col("YearBuilt")) <= 10)
    .then(pl.lit("New"))
    .when((pl.col("YrSold") - pl.col("YearBuilt")) <= 30)
    .then(pl.lit("Mid"))
    .otherwise(pl.lit("Old"))
    .alias("HouseAgeCategory")
)

display(train_df.select("HouseAgeCategory").head())


HouseAgeCategory
str
"""New"""
"""Old"""
"""New"""
"""Old"""
"""New"""


### 集約処理
`group_by` : 特定の列でグループ化する
* `agg` : 集約関数を適用する
  * `sum` : 合計
  *  `mean` : 平均
  *  `min` : 最小値
  *  `max` : 最大値
  *  `count` : カウント
  *  `first` : 最初の値
  *  `last` : 最後の値
  *  `median` : 中央値
  *  `std` : 標準偏差
  *  `var` : 分散
  *  `quantile` : 分位数
  *  `n_unique` : ユニークな値の数

In [62]:
result = (
    train_df.group_by("BsmtQual")
    .agg(
        [
            pl.mean("SalePrice").alias("SalePrice_mean"),
            pl.std("SalePrice").alias("SalePrice_std"),
            pl.min("SalePrice").alias("SalePrice_min"),
            pl.max("SalePrice").alias("SalePrice_max"),
            pl.count("SalePrice").alias("SalePrice_count"),
            pl.sum("SalePrice").alias("SalePrice_sum"),
        ]
    )
    .sort("SalePrice_mean")
)

display(result)


BsmtQual,SalePrice_mean,SalePrice_std,SalePrice_min,SalePrice_max,SalePrice_count,SalePrice_sum
str,f64,f64,i64,i64,u32,i64
"""NA""",105652.891892,29278.694797,39300,198500,37,3909157
"""Fa""",115692.028571,34469.667278,61000,206900,35,4049221
"""TA""",140759.818182,43483.265311,34900,475000,649,91353122
"""Gd""",202688.478964,58092.364284,75500,538000,618,125261480
"""Ex""",327041.041322,113563.237392,123500,755000,121,39571966


### 結合
`join` : データフレームを結合する
* `inner` : 内部結合
* `left` : 左外部結合
* `right` : 右外部結合
* `outer` : 外部結合
* `cross` : クロス結合

In [ ]:
# 集約処理した内容を結合する
agg_df = (
    train_df.group_by("BsmtQual")
    .agg(
        [
            pl.mean("BsmtFinSF1").alias("BsmtFinSF1_mean"),
            pl.std("BsmtFinSF1").alias("BsmtFinSF1_std"),
            pl.min("BsmtFinSF1").alias("BsmtFinSF1_min"),
            pl.max("BsmtFinSF1").alias("BsmtFinSF1_max"),
            pl.count("BsmtFinSF1").alias("BsmtFinSF1_count"),
            pl.sum("BsmtFinSF1").alias("BsmtFinSF1_sum"),
        ]
    )
    .sort("BsmtFinSF1_mean")
)

# train_dfとagg_dfを結合する(leftとすることで、train_dfのBsmtQualに対してagg_dfのBsmtQualを結合する)
train_df = train_df.join(agg_df, on="BsmtQual", how="left")
display(train_df.select(["BsmtQual", "BsmtFinSF1", "BsmtFinSF1_mean"]).head())


BsmtQual,BsmtFinSF1,BsmtFinSF1_mean
str,i64,f64
"""Gd""",706,472.501618
"""Gd""",978,472.501618
"""Gd""",486,472.501618
"""TA""",216,381.568567
"""Gd""",655,472.501618


### 遅延評価

`lazy` : 遅延評価を行う  
`collect` : 遅延評価を実行する

In [48]:
result_df = (
    train_df.lazy()  # lazyをつけることで、遅延評価を行う
    .group_by("BsmtQual")
    .agg(
        [
            pl.mean("SalePrice").alias("SalePrice_mean"),
            pl.std("SalePrice").alias("SalePrice_std"),
            pl.min("SalePrice").alias("SalePrice_min"),
            pl.max("SalePrice").alias("SalePrice_max"),
            pl.count("SalePrice").alias("SalePrice_count"),
            pl.sum("SalePrice").alias("SalePrice_sum"),
        ]
    )
    .sort("SalePrice_mean")
    .select(["BsmtQual", "SalePrice_mean"])
    .with_columns(
        [
            pl.when(pl.col("SalePrice_mean") < 200000)
            .then(pl.lit("Low"))
            .when(pl.col("SalePrice_mean") < 300000)
            .then(pl.lit("Middle"))
            .otherwise(pl.lit("High"))
            .alias("SalePriceCategory"),
        ]
    )
    .collect()  # collect()で遅延していた処理を実行する
)


display(result_df)


BsmtQual,SalePrice_mean,SalePriceCategory
str,f64,str
"""NA""",105652.891892,"""Low"""
"""Fa""",115692.028571,"""Low"""
"""TA""",140759.818182,"""Low"""
"""Gd""",202688.478964,"""Middle"""
"""Ex""",327041.041322,"""High"""
